In [2]:
import sys
sys.path
sys.path.append("/home/haoqi.whq/llm-inference/LoRA")

In [3]:
from src.model import GPT2Config, GPT2LMModel
import torch
from loralib import PruneLayer
import loralib as lora
import copy
import importlib
from torchsummary import summary


In [8]:
state_dict = torch.load("./pretrained_checkpoints/gpt2-medium-pytorch_model.bin")
# print(state_dict)
def split_attention_weights(state_dict):
    state_dict_tmp = copy.deepcopy(state_dict)
    attn_lists = ["q_attn", "k_attn", "v_attn"]
    for n, p in state_dict.items():
        if "c_attn" in n:
            module, name = n.split("c_attn.")
            dim = 0 if name == "bias" else 1
            split_size = p.shape[0] if name == "weight" else p.shape[0] // 3
            params = p.split(split_size, dim=dim)
            for i, attn in enumerate(attn_lists):
                # print(module+attn+"."+name)
                state_dict_tmp[module+attn+"."+name] = params[i]
    return state_dict_tmp

state_dict_tmp = split_attention_weights(state_dict)
print(state_dict_tmp["h.0.attn.q_attn.weight"].shape)

torch.Size([1024, 1024])


In [3]:
from torch.utils.data import DataLoader
from src.data_utils import FT_Dataset

train_data = "./data/e2e/train.jsonl"
train_batch_size = 2
seq_len = 512
obj = "clm"
label_smooth = 0.1
train_data = FT_Dataset(
    train_data, train_batch_size, seq_len, joint_lm= obj == "jlm"
)

train_loader = DataLoader(
    train_data,
    batch_size=train_batch_size,
    num_workers=0,
    shuffle=False,
    pin_memory=False,
    drop_last=True,
    # sampler=torch(
    #     train_data, seed=10
    # ),
)
print(type(train_loader))
# data = train_loader[0]

for i, data in enumerate(train_loader):
    data = {key: value for key, value in data.items()}
    _input = data["input"]
    _target = data["target"]
    _msk = data["mask"]

    print(_input.shape)
    print(_target.shape)
    print(_msk.shape)
    break

<class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 512])


In [13]:
# medium GPT2
config = GPT2Config(
    n_embd=1024,
    n_layer=24,
    n_head=16,
    lora_attn_dim=4,
    lora_attn_alpha=32,
    lora_dropout=0.1,
    enable_mlp=True,
    enable_wo=True,
    enable_wq=True,
    enable_wk=True,
    enable_wv=True,
)

lm_net = GPT2LMModel(config)

# summary(lm_net, _input, lm_labels=_target, lm_mask=_msk, label_smooth=label_smooth, depth=6)
# print(lm_net.state_dict())
# transformer.h.0.attn.q_atten.lora_B
# lm_head.decoder.weight
# print("loading model pretrained weight.")
lm_net.load_lora_weight("./pretrained_checkpoints/gpt2-medium-pytorch_model.bin", torch.load("/home/haoqi.whq/llm-inference/LoRA/examples/NLG/trained_models/GPT2_M/e2e/model.total.26290.pt"))


O Attention using LoRA: PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
)
MLP using LoRA: PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
), PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
)


In [14]:
lm_net.load_weight(torch.load("/home/haoqi.whq/llm-inference/LoRA/examples/NLG/trained_models/GPT2_M/e2e/model.lora.1000.pt"))
for m in lm_net.modules():
    if isinstance(m, PruneLayer):
        print(m, m.complexity())

PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64219.3633, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64219.4727, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64217.6719, grad_fn=<MulBackward0>)
PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64218.4102, grad_fn=<MulBackward0>)
PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(160547.5781, grad_fn=<MulBackward0>)
PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(160545.0938, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64219.2461, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024,

In [16]:
torch.load("/home/haoqi.whq/llm-inference/LoRA/examples/NLG/trained_models/GPT2_M/e2e/model.lora.3000.pt")

{'model_state_dict': {'module.module.transformer.h.0.attn.q_attn.lora_A': tensor([[-0.0169,  0.0340, -0.0044,  ..., -0.0014, -0.0243,  0.0151],
          [-0.0098,  0.0387, -0.0060,  ..., -0.0231, -0.0067,  0.0338],
          [-0.0218,  0.0092,  0.0091,  ...,  0.0259,  0.0106,  0.0037],
          [ 0.0028,  0.0141,  0.0129,  ..., -0.0095,  0.0046,  0.0310]],
         device='cuda:0'),
  'module.module.transformer.h.0.attn.q_attn.lora_B': tensor([[-0.0087, -0.0019, -0.0061, -0.0050],
          [-0.0037, -0.0040, -0.0110, -0.0107],
          [ 0.0045, -0.0016,  0.0049, -0.0017],
          ...,
          [ 0.0036,  0.0032, -0.0021,  0.0028],
          [-0.0061, -0.0053, -0.0028, -0.0053],
          [-0.0071, -0.0045, -0.0093, -0.0039]], device='cuda:0'),
  'module.module.transformer.h.0.attn.q_attn.lora_scaling': tensor(7.6322, device='cuda:0'),
  'module.module.transformer.h.0.attn.k_attn.lora_A': tensor([[-0.0047, -0.0116,  0.0114,  ..., -0.0008,  0.0190,  0.0638],
          [ 0.0388,  

In [5]:
lora.mark_only_lora_as_trainable(lm_net)
for m in lm_net.modules():
    if isinstance(m, PruneLayer) and hasattr(m, "lora_scaling"):
        print(m.lora_scaling)

for n, p in lm_net.named_parameters():
    if "lora_" in n:
        print(n)
# for n, p in lm_net.named_parameters():
#     if "lora" in n:
#         print(n)

Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.

## Efficiency Comparison
The baseline is all-in-tee.
The usage of lora in tee is to reduce the computation complexity in tee, consequently the runtime.

qkv statistics
- GPU linear time: 0.1957416534423828 ms
- CPU linear time: 9.22703742980957 ms
- CPU lora time: 1.2764930725097656 ms

mlp statistics
- GPU linear time: 0.3578662872314453 ms
- CPU linear time: 69.11993026733398 ms
- CPU lora time: 5.603551864624023 ms

In [25]:
import time

B = 8
SEQ_LEN = 512
r = 4

# [base, medium, large]
MODEL = "medium"

if MODEL == "base":
    n_embd = 768
    n_head = 12
elif MODEL == "medium":
    n_embd = 1024
    n_head = 16
elif MODEL == "large":
    n_embd = 1280
    n_head = 20
else:
    raise NotImplementedError(f"Model {MODEL} not supported.")

# GPU linear time: 0.19550323486328125 ms
# CPU linear time: 9.151220321655273 ms
# CPU lora time: 1.3489723205566406 ms
def attention_qkv():
    x = torch.rand(size=(B, SEQ_LEN, n_embd))
    
    # GPU computations
    x = x.cuda()
    gpu_linear = torch.nn.Linear(n_embd, n_embd).cuda()
    start = time.time()
    gpu_res = gpu_linear(x)
    end = time.time()
    print(f"GPU linear time: {(end-start)*1000} ms")

    x = x.cpu()
    cpu_linear = gpu_linear.cpu()
    start = time.time()
    cpu_res = cpu_linear(x)
    end = time.time()
    print(f"CPU linear time: {(end-start)*1000} ms")

    # cpu lora computations
    x = x.cpu()
    lora_A = torch.nn.Parameter(torch.rand((r, n_embd)))
    lora_B = torch.nn.Parameter(torch.rand((n_embd, r)))
    start = time.time()
    cpu_lora = x @ lora_A.transpose(0, 1) @ lora_B.transpose(0, 1)
    end = time.time()
    print(f"CPU lora time: {(end-start)*1000} ms")

# Same as qkv
def attention_o():
    x = torch.rand(size=(B, SEQ_LEN, n_embd))
    
    # GPU computations
    x = x.cuda()
    gpu_linear = torch.nn.Linear(n_embd, n_embd).cuda()
    start = time.time()
    gpu_res = gpu_linear(x)
    end = time.time()
    print(f"GPU linear time: {(end-start)*1000} ms")

    x = x.cpu()
    cpu_linear = gpu_linear.cpu()
    start = time.time()
    cpu_res = cpu_linear(x)
    end = time.time()
    print(f"CPU linear time: {(end-start)*1000} ms")

    # cpu lora computations
    x = x.cpu()
    lora_A = torch.nn.Parameter(torch.rand((r, n_embd)))
    lora_B = torch.nn.Parameter(torch.rand((n_embd, r)))
    start = time.time()
    cpu_lora = x @ lora_A.transpose(0, 1) @ lora_B.transpose(0, 1)
    end = time.time()
    print(f"CPU lora time: {(end-start)*1000} ms")

def attention_mlp():
    x = torch.rand(size=(B, SEQ_LEN, n_embd))
    
    # GPU computations
    x = x.cuda()
    gpu_linear1 = torch.nn.Linear(n_embd, n_embd*4).cuda()
    gpu_linear2 = torch.nn.Linear(n_embd*4, n_embd).cuda()
    start = time.time()
    gpu_res = gpu_linear2(gpu_linear1(x))
    end = time.time()
    print(f"GPU linear time: {(end-start)*1000} ms")

    x = x.cpu()
    cpu_linear1 = gpu_linear1.cpu()
    cpu_linear2 = gpu_linear2.cpu()
    start = time.time()
    cpu_res = cpu_linear2(cpu_linear1(x))
    end = time.time()
    print(f"CPU linear time: {(end-start)*1000} ms")

    # cpu lora computations
    x = x.cpu()
    lora_A = torch.nn.Parameter(torch.rand((r, n_embd)))
    lora_B = torch.nn.Parameter(torch.rand((n_embd*4, r)))

    lora_A2 = torch.nn.Parameter(torch.rand((r, n_embd*4)))
    lora_B2 = torch.nn.Parameter(torch.rand((n_embd, r)))
    start = time.time()
    cpu_lora = x @ lora_A.transpose(0, 1) @ lora_B.transpose(0, 1)
    cpu_lora_res = cpu_lora @ lora_A2.transpose(0, 1) @ lora_B2.transpose(0, 1)
    end = time.time()
    print(f"CPU lora time: {(end-start)*1000} ms")

print(f"qkv statistics")
attention_qkv()

print(f"mlp statistics")
attention_mlp()

qkv statistics
GPU linear time: 0.1957416534423828 ms
CPU linear time: 9.22703742980957 ms
CPU lora time: 1.2764930725097656 ms
mlp statistics
GPU linear time: 0.3578662872314453 ms
CPU linear time: 69.11993026733398 ms
CPU lora time: 5.603551864624023 ms
